In [ ]:
 !pip install transformers

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentimentAnalyser = SentimentIntensityAnalyzer()

nltk.download('stopwords')
from nltk.corpus import stopwords

import unicodedata

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
classifier = pipeline("zero-shot-classification",
                      device=0
                      )

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

## **Give access to your Google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_full = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_final_cleaned.csv')

In [ ]:
df_full.head()

,Unnamed: 0,date,id,content,replyCount,retweetCount,likeCount,quoteCount,lang,hashtags,mentionedUsers,user.id,user.location,place.country,place.name,coordinates.longitude,coordinates.latitude,keyword_used,cleaned_content
0,0,2021-12-20 03:52:10+00:00,1472776815236157441,This is just Pathetic \nI still think she’s ta...,1.0,0,4.0,0.0,en,['Counselling'],False,1360854673628913665,"Ballarat, Victoria",Australia,Ballarat,143.780231,-37.659717,#counselling,this is just pathetic i still think she s taki...
1,1,2021-12-18 21:05:07+00:00,1472311992530587649,What a view to welcome in Sunday Selfcare day....,0.0,0,4.0,0.0,en,"['selfcare', 'sunrise', 'metier', 'counselling...",False,104040029,Australia,Australia,Tasmania,143.818923,-43.740510,#counselling,what a view to welcome in sunday selfcare day ...
2,2,2021-12-16 00:37:47+00:00,1471278347145474048,Day 4\n\nWe work hard and when we achieve goal...,0.0,1,2.0,0.0,en,"['possessions', 'christmasgifts', 'stress', 'm...",False,104040029,Australia,Australia,Sydney,150.520929,-34.118347,#counselling,day 4 we work hard and when we achieve goals w...
3,3,2021-12-14 00:59:44+00:00,1470559094679293954,I'm doing 12 things that I'm grateful for coun...,1.0,0,1.0,0.0,en,"['grateful', 'gratitude', 'counselling', 'self...",False,104040029,Australia,Australia,Sydney,150.520929,-34.118347,#counselling,i am doing 12 things that i am grateful for co...
4,4,2021-12-13 07:13:26+00:00,1470290751330291713,I'm doing 12 things that I'm grateful for.\nIt...,1.0,1,2.0,0.0,en,"['gratitude', 'counselling', 'mentalhealth']",False,104040029,Australia,Australia,Sydney,150.520929,-34.118347,#counselling,i am doing 12 things that i am grateful for it...


## **Specify the number of rows here:**
- present 0th to 10th row.


In [ ]:
df = df_full.iloc[60000:90000].copy()

In [ ]:
strings = df.cleaned_content.values
strings[:5]

array(['microsoft open sources its coronavirus threat data for security researchers  t co 61vjgvnjq1',
       'stale beer reaching use by date goes to waste as pubs and clubs remain closed due to coronavirus restrictions abc news  t co y7ytder5k8',
       ' presssec realdonaldtrump in the true tradition of the last three press secretaries a shameless unmitigated liar donald trump won t allow the coronavirus to come to the us need to have a few lying tutorials from the queen of deceit blondie the peerless sarah sanders ',
       'loved this article by alice cuddy of life back when coronavirus the last normal photo on your phone  t co lgvkdznwti',
       'white house says no matter how you look at it we are leading the world in testing coronavirus i can only find 35 countries that are doing more tests per capita am i not looking properly '],
      dtype=object)

In [ ]:
sequence = list(strings)
sequence[:5]

['microsoft open sources its coronavirus threat data for security researchers  t co 61vjgvnjq1',
 'stale beer reaching use by date goes to waste as pubs and clubs remain closed due to coronavirus restrictions abc news  t co y7ytder5k8',
 ' presssec realdonaldtrump in the true tradition of the last three press secretaries a shameless unmitigated liar donald trump won t allow the coronavirus to come to the us need to have a few lying tutorials from the queen of deceit blondie the peerless sarah sanders ',
 'loved this article by alice cuddy of life back when coronavirus the last normal photo on your phone  t co lgvkdznwti',
 'white house says no matter how you look at it we are leading the world in testing coronavirus i can only find 35 countries that are doing more tests per capita am i not looking properly ']

# **Multi-Class Classification using Zero Shot Classifier**
### **Classes used: ``"panic", "anger", "sad", 'happy', "brave"``**

In [ ]:
candidate_labels = ["panic", "anger", "sad", 'happy', "brave" ]

In [ ]:
len(sequence)

29999

In [ ]:
labels_zero_shot = []
respective_scores = []
for i in range(len(sequence)):
  label_ = classifier(sequence[i], candidate_labels)
  labels_zero_shot.append(label_['labels'])
  respective_scores.append(label_['scores'])
pass
print(len(respective_scores), len(labels_zero_shot))

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1077: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


29999 29999


In [ ]:
new_df = pd.DataFrame()
new_df['tweet_content'] = df.cleaned_content
new_df['labels_zero_shot'] = labels_zero_shot
new_df['respective_scores'] = respective_scores

new_df.head()

,tweet_content,labels_zero_shot,respective_scores
0,this is just pathetic i still think she s taki...,"[sad, anger, panic, brave, happy]","[0.9776253700256348, 0.010648265480995178, 0.0..."
1,what a view to welcome in sunday selfcare day ...,"[happy, brave, panic, anger, sad]","[0.971422553062439, 0.020104505121707916, 0.00..."
2,day 4 we work hard and when we achieve goals w...,"[happy, brave, panic, sad, anger]","[0.41082775592803955, 0.31598541140556335, 0.1..."
3,i am doing 12 things that i am grateful for co...,"[happy, brave, panic, sad, anger]","[0.581499457359314, 0.3123384118080139, 0.0440..."
4,i am doing 12 things that i am grateful for it...,"[brave, sad, panic, happy, anger]","[0.4356525242328644, 0.2900204360485077, 0.145..."


In [ ]:
best_labels_scores = []
best_labels_one_shot = []
for i,j in zip(new_df['labels_zero_shot'],new_df['respective_scores']):
  a = i,j
  best_labels_scores.append(a[1][0])
  best_labels_one_shot.append(a[0][0])
new_df['best_labels_scores'] = best_labels_scores
new_df['best_labels'] = best_labels_one_shot

In [ ]:
new_df.head()

,tweet_content,labels_zero_shot,respective_scores,best_labels_scores,best_labels
0,this is just pathetic i still think she s taki...,"[sad, anger, panic, brave, happy]","[0.9776253700256348, 0.010648265480995178, 0.0...",0.977625,sad
1,what a view to welcome in sunday selfcare day ...,"[happy, brave, panic, anger, sad]","[0.971422553062439, 0.020104505121707916, 0.00...",0.971423,happy
2,day 4 we work hard and when we achieve goals w...,"[happy, brave, panic, sad, anger]","[0.41082775592803955, 0.31598541140556335, 0.1...",0.410828,happy
3,i am doing 12 things that i am grateful for co...,"[happy, brave, panic, sad, anger]","[0.581499457359314, 0.3123384118080139, 0.0440...",0.581499,happy
4,i am doing 12 things that i am grateful for it...,"[brave, sad, panic, happy, anger]","[0.4356525242328644, 0.2900204360485077, 0.145...",0.435653,brave


# **Sentiment Analysis ``(Positive and Negative)`` using Zero Shot Classifier**

In [ ]:
sentiment_labels = ['positive','negative']

In [ ]:
sentiments_zero_shot = []
respective_scores = []
for i in range(len(sequence)):
  label_ = classifier(sequence[i], sentiment_labels)
  sentiments_zero_shot.append(label_['labels'])
  respective_scores.append(label_['scores'])

print(len(respective_scores), len(sentiments_zero_shot))

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1077: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


10 10


In [ ]:
sentiments_zero_shot[:2]

[['negative', 'positive'], ['positive', 'negative']]

In [ ]:
best_sentiments_scores = []
sentiment = []
for i,j in zip(sentiments_zero_shot,respective_scores):
  a = i,j
  best_sentiments_scores.append(a[1][0])
  sentiment.append(a[0][0])
new_df['sentiments_one_shot_classifier_score'] = best_sentiments_scores
new_df['sentiments_one_shot_classifier'] = sentiment

In [ ]:
new_df.head()

,tweet_content,labels_zero_shot,respective_scores,best_labels_scores,best_labels,sentiments_one_shot_classifier_score,sentiments_one_shot_classifier
0,this is just pathetic i still think she s taki...,"[sad, anger, panic, brave, happy]","[0.9776253700256348, 0.010648265480995178, 0.0...",0.977625,sad,0.998147,negative
1,what a view to welcome in sunday selfcare day ...,"[happy, brave, panic, anger, sad]","[0.971422553062439, 0.020104505121707916, 0.00...",0.971423,happy,0.995800,positive
2,day 4 we work hard and when we achieve goals w...,"[happy, brave, panic, sad, anger]","[0.41082775592803955, 0.31598541140556335, 0.1...",0.410828,happy,0.936278,positive
3,i am doing 12 things that i am grateful for co...,"[happy, brave, panic, sad, anger]","[0.581499457359314, 0.3123384118080139, 0.0440...",0.581499,happy,0.962516,positive
4,i am doing 12 things that i am grateful for it...,"[brave, sad, panic, happy, anger]","[0.4356525242328644, 0.2900204360485077, 0.145...",0.435653,brave,0.551461,positive


In [ ]:
new_df.sentiments_one_shot_classifier.value_counts()

positive    9
negative    1
Name: sentiments_one_shot_classifier, dtype: int64

## **Sentiment Analysis using VADER ``(Positive and Negative)``**

### **Pre-Processing Text**

In [ ]:
import re

stop_words=set( stopwords.words('english') )



"""Functions to clean text using Regex"""
ip_addr_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def regex_cleaning(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"[^a-zA-Z0-9]+", " ", phrase)
    phrase = re.sub('[\(\[].*?[\)\]]', ' ', phrase)
    phrase = unicodedata.normalize('NFKD', phrase).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    phrase = re.sub(r"\r\n", "", phrase)            # Removing additional line
    phrase = re.sub(r"\n", "", phrase)              # Removing additional line
    phrase = re.sub(r"\S*@\S*\s?", "", phrase)      # Removing email-addresses
    phrase = re.sub(r'http\S+', '', phrase)         # Removing website links
    phrase = re.sub(ip_addr_regex, "", phrase)      # Removing IP address link.
    phrase = emoji_pattern.sub(r'', phrase)         # Removing Emojis

    phrase_final = " ".join([word for word in phrase.split() if word not in stop_words and len(word) > 3])
    return phrase_final

import spacy
import gensim

def gen_words(text):
    final = []
    # for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    for word in new:
      if len(word) > 3:
        final.append(word)
    # print(final)
    return (" ".join(final))



def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm")

    texts_out = []
    for text in texts:

      doc = nlp(text)
      new_text = []
      for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
      final = " ".join(new_text)
      texts_out.append(final)
    return (texts_out)



In [ ]:
%%time
new_df['tweet_content'] = new_df['tweet_content'].apply(regex_cleaning)

new_df['tweet_content'] = lemmatization(new_df.tweet_content)

"""Sentiment Analysis using Vader"""
new_df['sentiment_via_VADER'] = new_df['tweet_content'].apply(lambda review: "positive" if sentimentAnalyser.polarity_scores(review)['compound'] > 0.5 else "negative")

CPU times: user 747 ms, sys: 55.8 ms, total: 802 ms
Wall time: 883 ms


In [ ]:
df.head()

,Unnamed: 0,date,id,content,replyCount,retweetCount,likeCount,quoteCount,lang,hashtags,mentionedUsers,user.id,user.location,place.country,place.name,coordinates.longitude,coordinates.latitude,keyword_used,cleaned_content
0,0,2021-12-20 03:52:10+00:00,1472776815236157441,This is just Pathetic \nI still think she’s ta...,1.0,0,4.0,0.0,en,['Counselling'],False,1360854673628913665,"Ballarat, Victoria",Australia,Ballarat,143.780231,-37.659717,#counselling,this is just pathetic i still think she s taki...
1,1,2021-12-18 21:05:07+00:00,1472311992530587649,What a view to welcome in Sunday Selfcare day....,0.0,0,4.0,0.0,en,"['selfcare', 'sunrise', 'metier', 'counselling...",False,104040029,Australia,Australia,Tasmania,143.818923,-43.740510,#counselling,what a view to welcome in sunday selfcare day ...
2,2,2021-12-16 00:37:47+00:00,1471278347145474048,Day 4\n\nWe work hard and when we achieve goal...,0.0,1,2.0,0.0,en,"['possessions', 'christmasgifts', 'stress', 'm...",False,104040029,Australia,Australia,Sydney,150.520929,-34.118347,#counselling,day 4 we work hard and when we achieve goals w...
3,3,2021-12-14 00:59:44+00:00,1470559094679293954,I'm doing 12 things that I'm grateful for coun...,1.0,0,1.0,0.0,en,"['grateful', 'gratitude', 'counselling', 'self...",False,104040029,Australia,Australia,Sydney,150.520929,-34.118347,#counselling,i am doing 12 things that i am grateful for co...
4,4,2021-12-13 07:13:26+00:00,1470290751330291713,I'm doing 12 things that I'm grateful for.\nIt...,1.0,1,2.0,0.0,en,"['gratitude', 'counselling', 'mentalhealth']",False,104040029,Australia,Australia,Sydney,150.520929,-34.118347,#counselling,i am doing 12 things that i am grateful for it...


In [ ]:
new_df.sentiment_via_VADER.value_counts()

negative    5
positive    5
Name: sentiment_via_VADER, dtype: int64

## **Please update the row numbers in the row_numbers variable in the same-way.**

In [ ]:
row_numbers = '60000_to_90000'
new_df.to_csv(f'tweets_final_cleaned_{row_numbers}.csv')